In [2]:
from multiprocessing import Process,Queue
import os

# data_file = "github"
# data_file = "test"
data_file = "wiki"

INF = 1e9
SourceNode = 1
class HyperNode:
    def __init__(self,n_id):
        self.n_id = n_id
        self.dist = INF 
        self.edges = []
        
    def AddEdge(self,edge):
        self.edges.append(edge)
        
    def GetHyperEdges(self):
        return self.edges
    
    def GetValue(self):
        return self.dist
    
                    
class HyperEdge:
    def __init__(self,e_id):
        self.e_id = e_id
        self.nodes = []
        
    def AddNode(self,node):
        self.nodes.append(node)
    
    def GetHyperNodes(self):
        return self.nodes
    
    def getValue(self):
        return 1
    
# def solve():
#     while 
#         for n_id,node in Node.items():
#             node.compute(msg[n_id])
        


# def load_data():
#     with open("./test_data/partition_method.txt") as file :
#         for line in file:
#             v_id,par_id = line[0:-1].split(" ")
#             Part[v_id] = par_id
    
#     with open("./test_data/github_vertex_info.txt") as file:
#         for line in file:
#             data = line[0:-1].split(" ")
#             v_id = int(data[0])
#             Node[v_id] = HyperNode(v_id)
#             for e_id in data[1:]:
#                 Node[v_id].AddEdge(int(e_id))
                
#     with open("./test_data/github_edge_info.txt") as file:
#         for line in file:
#             data = line[0:-1].split(" ")
#             e_id = int(data[0])
#             Edge[e_id] = HyperEdge(e_id)
#             for n_id in data[1:]:
#                 Edge[e_id].AddNode(int(n_id))

        
    

In [3]:
class Master:
    def __init__(self,p):
        self.p = p
        self.Node = {}
        self.Edge = {}
        self.Part = {}
        self.Worker = []
        for i in range(p) :
            self.Worker.append(Worker(i))
        
        self.result = {}

    def load_data(self):
        with open("./test_data/"+data_file+"/partition_method.txt") as file :  # (n_id,p_id)
            for line in file:
                v_id,par_id = line[0:-1].split(" ")
                v_id,par_id = int(v_id),int(par_id)
                self.Part[v_id] = par_id
                    
        with open("./test_data/"+data_file+"/vertex_info.txt") as file:  # v_id, e1_id, e2_id, ...
            for line in file:
                data = line[0:-1].split(" ")
                v_id = int(data[0])
                self.Node[v_id] = HyperNode(v_id)
                for e_id in data[1:]:
                    self.Node[v_id].AddEdge(int(e_id))
                    
        with open("./test_data/"+data_file+"/edge_info.txt") as file:   # e_id, v1_id, v2_id,...
            for line in file:
                data = line[0:-1].split(" ")
                e_id = int(data[0])
                self.Edge[e_id] = HyperEdge(e_id)
                for n_id in data[1:]:
                    self.Edge[e_id].AddNode(int(n_id))

                    
    def distribute_node(self,node,w_id):
        if node.n_id in self.Worker[w_id].Node : return # nonsence
        self.Worker[w_id].receive_node(node)
#         self.Worker[w_id].Node[n_id] = self.Node[n_id]
    
    def distribute_edge(self,edge,w_id):
        if edge.e_id in self.Worker[w_id].Edge : return # avoid repeat send
#         self.Worker[w_id].Edge[e_id] = self.Edge[e_id]
        self.Worker[w_id].receive_edge(edge)
    
    def distribute_part(self):
        for worker in self.Worker:
            worker.receive_part(self.Part)
        
    def distribute(self):
        self.distribute_part()
                
        for n_id,node in self.Node.items():
            self.distribute_node(node,self.Part[n_id])
            for e_id in node.edges:
                self.distribute_edge(self.Edge[e_id],self.Part[n_id])
                
    def get_result(self):
        for worker in self.Worker:
            for n_id,node in worker.Node.items():
                self.result[n_id] = node.dist
                
                
                   
    def compute(self):
        turn = 0
        msg_cnt = 1
        while msg_cnt != 0 :
            turn += 1
            msg_cnt = 0
            
            for worker in self.Worker:   
                worker.communicate_rece()
            print("communicate rece finished!")
            
            for worker in self.Worker:
                msg_cnt += worker.compute()          
            print(turn," msg_cnt:",msg_cnt)
            print("compute finished!")
            
            for worker in self.Worker:
                worker.communicate_send()
            print("comunicate send finished!")
            
            self.get_result()
            dic = {}
            for key in range(5): 
                dic[key] = 0
            for key,value in self.result.items():
                if value not in dic :
                    dic[value] = 0
                dic[value] += 1
            for key in range(5):
                print(key,":",dic[key])

    
    def init(self):
        self.Worker[self.Part[SourceNode ]].recvBuf.append((1,0))
            
            
                    
class Worker:
    def __init__(self,w_id):
        self.w_id = w_id
        self.Node = {}
        self.Edge = {}
        self.Part = {}
        self.msg = {}
        self.sendBuf = []
        self.recvBuf = []
        filepath = "./pipeline/"+str(self.w_id)
        open(filepath,"w")
        
    def cal_node(self,node):
        msgs = self.msg[node.n_id]
        mindist = INF 
        mindist = min(msgs)
        sendMessage = []
        if mindist < node.dist : 
            node.dist = mindist
            for e_id in node.GetHyperEdges():
                hyperEdge = self.Edge[e_id]
                for n_id in hyperEdge.GetHyperNodes():
                    sendMessage.append((n_id,mindist+hyperEdge.getValue()))
#                     SendMessageTo(self.v_id,target,mindist+hyperEdge.getValue())
        return sendMessage
    
    def receive_node(self,Node):
        self.Node[Node.n_id] = Node
        
    def receive_edge(self,Edge):
        self.Edge[Edge.e_id] = Edge
        
    def receive_part(self,Part):
        self.Part = Part
    
    def compute(self):
        self.msg = {}
        for tar,value in self.recvBuf: # receive msg from previous superstep
            if tar not in self.msg :
                self.msg[tar] = []
            self.msg[tar].append(value)
        self.recvBuf = []
        
        msg_cnt = 0
        for n_id,node in self.Node.items():
            if n_id not in self.msg : continue
            msg = self.cal_node(node) # compute and get msg info
            msg_cnt += len(msg)
            for tar,value in msg:  # msg transfer judge
                if self.Part[tar] == self.w_id:
                    self.recvBuf.append((tar,value))
                else : 
                    self.sendBuf.append((tar,value))
        print("worker msg:",self.w_id," ",msg_cnt)
        return msg_cnt
    
    def communicate_send(self):
        filepath = "./pipeline/"
        for tar,value in self.sendBuf:
            with open(filepath+str(self.Part[tar]),"a") as file:
                file.write(str(tar)+" "+str(value)+"\n")
        self.sendBuf.clear()
                
    def communicate_rece(self):
        filepath = "./pipeline/"+str(self.w_id)
        with open(filepath,"r") as file:
            for line in file:
                tar,value =  line[0:-1].split(" ")
                tar,value = int(tar),int(value)
                self.recvBuf.append((tar,value))
                
        open(filepath,"w")
            
            
        
    
master = Master(4)
master.load_data()
master.distribute()
master.init()
master.compute()
for worker in master.Worker:
    print(len(worker.Part))
    print(len(worker.Node))

communicate rece finished!
worker msg: 0   0
worker msg: 1   0
worker msg: 2   1239
worker msg: 3   0
1  msg_cnt: 1239
compute finished!
comunicate send finished!
0 : 1
1 : 0
2 : 0
3 : 0
4 : 0
communicate rece finished!
worker msg: 0   3729795
worker msg: 1   1248733
worker msg: 2   583937
worker msg: 3   234818
2  msg_cnt: 5797283
compute finished!
comunicate send finished!
0 : 1
1 : 804
2 : 0
3 : 0
4 : 0
communicate rece finished!
worker msg: 0   6399175
worker msg: 1   4025476
worker msg: 2   2890321
worker msg: 3   1916982
3  msg_cnt: 15231954
compute finished!
comunicate send finished!
0 : 1
1 : 804
2 : 3758
3 : 0
4 : 0
communicate rece finished!
worker msg: 0   0
worker msg: 1   0
worker msg: 2   0
worker msg: 3   6
4  msg_cnt: 6
compute finished!
comunicate send finished!
0 : 1
1 : 804
2 : 3758
3 : 1
4 : 0
communicate rece finished!
worker msg: 0   0
worker msg: 1   0
worker msg: 2   0
worker msg: 3   0
5  msg_cnt: 0
compute finished!
comunicate send finished!
0 : 1
1 : 804
2 : 

In [3]:
result = {}
for worker in master.Worker:
    for n_id,node in worker.Node.items():
        result[n_id] = node.dist

In [4]:
dic = {}
for key,value in result.items():
    if value not in dic : 
        dic[value] = 0
    dic[value] += 1
for k,v in dic.items():
    print(k,":",v)

1 : 804
2 : 3758
0 : 1
1000000000.0 : 2
3 : 1
